In [1]:
in_path_labels = 'D:/Liver Segmentation Dataset/full_data_processing/dicom/labels'
out_path_labels = 'D:/Liver Segmentation Dataset/full_data_processing/dicom_groups/labels'
in_path_volumes = 'D:/Liver Segmentation Dataset/full_data_processing/dicom/images'
out_path_volumes = 'D:/Liver Segmentation Dataset/full_data_processing/dicom_groups/images'

In [1]:
from glob import glob
import shutil
import os
import dicom2nifti
import nibabel as nib
import numpy as np
import torch
import monai
from tqdm import tqdm
from monai.transforms import (
    Compose,
    EnsureChannelFirstD,
    LoadImaged,
    Resized,
    ToTensord,
    Spacingd,
    Orientationd,
    ScaleIntensityRanged,
    CropForegroundd,

)
from monai.data import DataLoader, Dataset, CacheDataset
from monai.utils import set_determinism

C:\ProgramData\anaconda3\envs\gpVenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Create groups of 64 dicom slices

In [3]:
patient_list = glob(in_path_volumes+ '/*')
print(patient_list)

patientss = [os.path.join(in_path_volumes,i) for i in os.listdir(in_path_volumes)]
print("+++++++++++++++++++++++++)))))))))))))))))**************____________")
print("patinetsss:",patientss)
print(len(patientss)) 

['D:/Liver Segmentation Dataset/full_data_processing/dicom/images\\vol_0', 'D:/Liver Segmentation Dataset/full_data_processing/dicom/images\\vol_1', 'D:/Liver Segmentation Dataset/full_data_processing/dicom/images\\vol_10', 'D:/Liver Segmentation Dataset/full_data_processing/dicom/images\\vol_100', 'D:/Liver Segmentation Dataset/full_data_processing/dicom/images\\vol_101', 'D:/Liver Segmentation Dataset/full_data_processing/dicom/images\\vol_102', 'D:/Liver Segmentation Dataset/full_data_processing/dicom/images\\vol_103', 'D:/Liver Segmentation Dataset/full_data_processing/dicom/images\\vol_104', 'D:/Liver Segmentation Dataset/full_data_processing/dicom/images\\vol_105', 'D:/Liver Segmentation Dataset/full_data_processing/dicom/images\\vol_106', 'D:/Liver Segmentation Dataset/full_data_processing/dicom/images\\vol_107', 'D:/Liver Segmentation Dataset/full_data_processing/dicom/images\\vol_108', 'D:/Liver Segmentation Dataset/full_data_processing/dicom/images\\vol_109', 'D:/Liver Segmen

In [17]:
NUMBER_OF_GROUPSLICES = 74
def create_dicom_groups(in_path, out_path):
    for patient in glob(in_path + '/*'):
        patient_name = os.path.basename(os.path.normpath(patient))
        number_folders = int(len(glob(patient+'/*'))/NUMBER_OF_GROUPSLICES)
        for i in range(number_folders):
            output_path_name = os.path.join(out_path,patient_name + '_' + str(i))
            os.mkdir(output_path_name)
            for j, file in enumerate(glob(patient+'/*')):
                if j == NUMBER_OF_GROUPSLICES :
                    break
                shutil.move(file, output_path_name)

In [18]:
create_dicom_groups(in_path_labels,out_path_labels)

In [19]:
create_dicom_groups(in_path_volumes,out_path_volumes)

## Convert the grouped dicom files into nifties

In [26]:
imagesgroups_in_path = 'D:/Liver Segmentation Dataset/full_data_processing/dicom_groups/images/*'
labelsgroups_in_path = 'D:/Liver Segmentation Dataset/full_data_processing/dicom_groups/labels/*'
out_path_niftisimages = 'D:/Liver Segmentation Dataset/full_data_processing/nifti_postprocessed_volumes/images'
out_path_niftislabels = 'D:/Liver Segmentation Dataset/full_data_processing/nifti_postprocessed_volumes/labels'

In [27]:
list_grouped_dicomimages = glob(imagesgroups_in_path)
list_grouped_dicomlabels = glob(labelsgroups_in_path)

In [28]:
for patient in list_grouped_dicomimages:
    patient_name = os.path.basename(os.path.normpath(patient))
    dicom2nifti.dicom_series_to_nifti(patient,os.path.join(out_path_niftisimages,patient_name + '.nii.gz') )

In [29]:
for patient in list_grouped_dicomlabels:
    patient_name = os.path.basename(os.path.normpath(patient))
    dicom2nifti.dicom_series_to_nifti(patient,os.path.join(out_path_niftislabels,patient_name + '.nii.gz') )

## Find Empty


In [34]:
# some groups having slices that doesn't have any label or segmentation which can affect the model accuracy
input_nifi_labels_path = 'D:/Liver Segmentation Dataset/full_data_processing/nifti_postprocessed_volumes/labels/*'
input_nifti_volumes_path = 'D:/Liver Segmentation Dataset/full_data_processing/nifti_postprocessed_volumes/images'


list_labels = glob(input_nifi_labels_path)

for label_path in list_labels:
    nifti_file = nib.load(label_path)
    fdata = nifti_file.get_fdata() #get_fdata retrieves the underlying 3D or 4D array data (depending on the scan type) as a floating-point NumPy array.
    np_unique = np.unique(fdata) #check the classes present in the segmentation mask to assure it is not an empty mask
    if len(np_unique) == 1:
        print("Empty_Label_Path:",label_path)
        
        # Derive the corresponding volume file path
        filename = os.path.basename(os.path.normpath(label_path))
        volume_path = os.path.join(input_nifti_volumes_path, filename)
        
        # Remove the label file
        if os.path.exists(label_path):
            os.remove(label_path)
            print(f"Deleted label file: {label_path}")

        # Remove the corresponding volume file
        if os.path.exists(volume_path):
            os.remove(volume_path)
            print(f"Deleted volume file: {volume_path}")

Empty_Label_Path: D:/Liver Segmentation Dataset/full_data_processing/nifti_postprocessed_volumes/labels\vol_100_0.nii.gz
Deleted label file: D:/Liver Segmentation Dataset/full_data_processing/nifti_postprocessed_volumes/labels\vol_100_0.nii.gz
Deleted volume file: D:/Liver Segmentation Dataset/full_data_processing/nifti_postprocessed_volumes/images\vol_100_0.nii.gz
Empty_Label_Path: D:/Liver Segmentation Dataset/full_data_processing/nifti_postprocessed_volumes/labels\vol_100_1.nii.gz
Deleted label file: D:/Liver Segmentation Dataset/full_data_processing/nifti_postprocessed_volumes/labels\vol_100_1.nii.gz
Deleted volume file: D:/Liver Segmentation Dataset/full_data_processing/nifti_postprocessed_volumes/images\vol_100_1.nii.gz
Empty_Label_Path: D:/Liver Segmentation Dataset/full_data_processing/nifti_postprocessed_volumes/labels\vol_100_2.nii.gz
Deleted label file: D:/Liver Segmentation Dataset/full_data_processing/nifti_postprocessed_volumes/labels\vol_100_2.nii.gz
Deleted volume file:

### Data Train Test Split

In [37]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Paths
nifti_dir = r'D:\Liver Segmentation Dataset\full_data_processing\nifti_postprocessed_volumes'
final_dir = r'D:\Liver Segmentation Dataset\full_data_processing\Data_Train_Test'

# Subfolders for images and labels
images_dir = os.path.join(nifti_dir, "images")
labels_dir = os.path.join(nifti_dir, "labels")

# Destination directories
train_vol_dir = os.path.join(final_dir, "TrainVol")
train_seg_dir = os.path.join(final_dir, "TrainSeg")
test_vol_dir = os.path.join(final_dir, "TestVol")
test_seg_dir = os.path.join(final_dir, "TestSeg")

# Create destination directories if they don't exist
os.makedirs(train_vol_dir, exist_ok=True)
os.makedirs(train_seg_dir, exist_ok=True)
os.makedirs(test_vol_dir, exist_ok=True)
os.makedirs(test_seg_dir, exist_ok=True) 

# List of files
image_files = sorted([f for f in os.listdir(images_dir) if f.endswith(".nii.gz")])
label_files = sorted([f for f in os.listdir(labels_dir) if f.endswith(".nii.gz")])

# Ensure correspondence between images and labels
assert len(image_files) == len(label_files), "Mismatch between images and labels."
assert all(os.path.splitext(img)[0] == os.path.splitext(lbl)[0] for img, lbl in zip(image_files, label_files)), "File names do not match."

# Split the data
train_images, test_images, train_labels, test_labels = train_test_split(
    image_files, label_files, test_size=0.2, random_state=42
)

# Move files to corresponding directories
for file in train_images:
    shutil.copy(os.path.join(images_dir, file), os.path.join(train_vol_dir, file))

for file in train_labels:
    shutil.copy(os.path.join(labels_dir, file), os.path.join(train_seg_dir, file))

for file in test_images:
    shutil.copy(os.path.join(images_dir, file), os.path.join(test_vol_dir, file))

for file in test_labels:
    shutil.copy(os.path.join(labels_dir, file), os.path.join(test_seg_dir, file))

print("Data split complete.")


Data split complete.


In [40]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [41]:
import monai
print(monai.__version__)



C:\ProgramData\anaconda3\envs\gpVenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1.4.0


### Calculate foreground and background pixels for DICELoss

In [2]:
def calculate_pixels(data):
    """
    Calculate the number of liver and non-liver pixels in a given dataset.

    Parameters:
        data (DataLoader): DataLoader containing the data.

    Returns:
        val (numpy.array): A 2-element array containing the number of liver and non-liver pixels, respectively.
    """
    val = np.zeros((1, 2))

    for batch in tqdm(data):
        batch_label = batch["seg"] != 0
        _, count = np.unique(batch_label, return_counts=True)

        if len(count) == 1:
            count = np.append(count, 0)
        val += count

    #print('The last values:', val)
    return val       


def calculate_weights(no_foreground_pixels, no_background_pixels):
    """
    This function calculates the weights for the cross entropy weighted loss function
    in order to place more  importance on the foreground class which is the liver
    since it is the class that we are trying to segment and the background class dominates
    most of the image
    """
    count_array = np.array([no_background_pixels,  no_foreground_pixels]) # pixel counts for the background and foreground classes are stored in an array.
    weights = count_array / count_array.sum() # calculate the proportions of each class
    adjusted_weights = 1/weights # adjust the weights to give more importance to the foreground class
    weights_normalized = adjusted_weights / adjusted_weights.sum() # normalize the adjusted weights
    return torch.tensor(weights_normalized, dtype=torch.float32)


In [3]:
def prepare_data_loader(in_dir, pixdim=(1.5,1.5,1.0), a_min = -200, a_max = 200, spatial_size = [128,128,128], cache = False):
    """
    This function is the main function for the preprocessing of the data, it 
    contains chaining of basic transforms that are applied to the data using the monai library.

    a_min and a_max are the min and max values of the intensity of the images, there
    value can be set by testing the images on ITKSnap and finding the suitable values.
    """

    #This function is used to control randomness in training to ensure reproducibility. By setting a specific seed, you can make sure that each run with the same parameters produces identical results. This is useful for debugging and validation.
    set_determinism(seed=0)
    
    #get the paths of the train and test volumes and segmentation masks
    path_train_volumes  = sorted(glob(os.path.join(in_dir, "TrainVol", "*.nii.gz")))
    path_train_segmentation = sorted(glob(os.path.join(in_dir, "TrainSeg", "*.nii.gz")))

    path_test_volumes = sorted(glob(os.path.join(in_dir, "TestVol", "*.nii.gz")))
    path_test_segmentation = sorted(glob(os.path.join(in_dir, "TestSeg", "*.nii.gz")))
     

    #create list of dictionaries for the train and test datasets
    train_files = [{"vol":image_train_path, "seg":segmentation_train_path} for image_train_path, segmentation_train_path in zip(path_train_volumes, path_train_segmentation)]
    test_files = [{"vol":image_test_path, "seg":segmentation_test_path} for image_test_path, segmentation_test_path in zip(path_test_volumes, path_test_segmentation)]
    
    """
    LoadImaged:

    Purpose: Loads image and label files into memory as dictionaries with keys specified in keys.
    Effect: Converts file paths to actual image data (e.g., numpy arrays).
    The files are decompressed and loaded as NumPy arrays internally using nibabel or a similar backend.
    After loading, the NumPy arrays replace the file paths in the dictionary under the same keys
    {"vol": numpy_array_for_volume, "seg": numpy_array_for_segmentation}

    After applying LoadImaged, the dataset or data loader now processes dictionaries where:

    The keys ("vol", "seg") map to the loaded NumPy arrays instead of file paths.

    EnsureChannelFirstD:

    Purpose: Reorders the channel dimension to be the first axis.
    Effect: Ensures images conform to the PyTorch standard shape [C, H, W] or [C, D, H, W].

    Spacingd:

    Purpose: Resamples the data to the desired voxel spacing (pixdim).
    Effect: Adjusts the physical resolution of the image, ensuring uniformity across datasets. Uses interpolation modes like "bilinear" for volumes and "nearest" for segmentation.

    Orientationd:

    Purpose: Reorients the image axes to follow the specified convention (axcodes, e.g., RAS: Right-Anterior-Superior).
    Effect: Standardizes image orientation, critical for aligning data during preprocessing.

    ScaleIntensityRanged:

    Purpose: Normalizes intensity values of the image within a specified range (a_min, a_max → b_min, b_max).
    Effect: Enhances numerical stability during training by scaling values to a uniform range.

    CropForegroundd:

    Purpose: Crops the image and segmentation to include only the foreground (non-zero intensity).
    Effect: Reduces computational overhead by focusing on regions of interest.

    Resized:

    Purpose: Resizes the image and segmentation to a consistent spatial size (spatial_size).
    Effect: Standardizes input dimensions, which is crucial for model training.

    ToTensord:

    Purpose: Converts numpy arrays or other formats to PyTorch tensors.
    Effect: Prepares the data for models using PyTorch.
    After Full Transform Pipeline: PyTorch tensors stored in the dictionary.


    """
    #monai transforms
    train_transforms = Compose([
        LoadImaged(keys=["vol", "seg"]),
        EnsureChannelFirstD(keys=["vol", "seg"]),
        Spacingd(keys=["vol", "seg"], pixdim=pixdim, mode=("bilinear", "nearest")),
        Orientationd(keys=["vol", "seg"], axcodes="RAS"),
        ScaleIntensityRanged(keys=["vol"], a_min=a_min, a_max=a_max, b_min=0.0, b_max=1.0, clip=True),
        CropForegroundd(keys=["vol", "seg"], source_key="vol"),
        Resized(keys=["vol", "seg"], spatial_size=spatial_size),
        ToTensord(keys=["vol", "seg"]),
    ])  

    test_transforms = Compose([
        LoadImaged(keys=["vol", "seg"]),
        EnsureChannelFirstD(keys=["vol", "seg"]),
        Spacingd(keys=["vol", "seg"], pixdim=pixdim, mode=("bilinear", "nearest")),
        Orientationd(keys=["vol", "seg"], axcodes="RAS"),
        ScaleIntensityRanged(keys=["vol"], a_min=a_min, a_max=a_max, b_min=0.0, b_max=1.0, clip=True),
        CropForegroundd(keys=['vol', 'seg'], source_key='vol'),
        Resized(keys=["vol", "seg"], spatial_size=spatial_size),
        ToTensord(keys=["vol", "seg"]),

    ])

    if cache:
        """
        Pre-applies transforms and caches the results, dramatically reducing per-epoch data preprocessing time.
        """
        #CacheDataset-->  The dataset is cached in memory for faster access during training.
        #DataLoader--> batching, shuffling, and parallel data loading. Works with CacheDataset and Dataset
        train_dataset = CacheDataset(data=train_files, transform=train_transforms, cache_rate=1.0)
        train_loader = DataLoader(train_dataset, batch_size=1)

        test_dataset = CacheDataset(data=test_files, transform=test_transforms, cache_rate=1.0)
        test_loader = DataLoader(test_dataset, batch_size=1)

        return train_loader, test_loader

    else:
        #Dataset-->  Each data sample is loaded and transformed on-the-fly for each iteration.
        #Saves memory by not caching the entire dataset in memory but is slower in training
        train_dataset = Dataset(data=train_files, transform=train_transforms)
        train_loader = DataLoader(train_dataset, batch_size=1)

        test_dataset = Dataset(data=test_files, transform=test_transforms)
        test_loader = DataLoader(test_dataset, batch_size=1)

        return train_loader, test_loader
    
data_dir = 'D:/Liver Segmentation Dataset/full_data_processing/Data_Train_Test' # path to the data directory, after preparing data(converting niftis into dicom and forming dicom groups then converting back to niftis )
data_loader = prepare_data_loader(data_dir, cache=True) # prepare data loader and cache it in GPU memory
train_loader, test_loader = data_loader
# Calculate pixel counts for the Dice loss
pixel_counts = calculate_pixels(train_loader)
print("Pixel_Counts:",pixel_counts)
# Calculate class weights for the Dice loss
class_weights = calculate_weights(pixel_counts[0][1], pixel_counts[0][0])
print("class_weights:",class_weights)

C:\ProgramData\anaconda3\envs\gpVenv\Lib\site-packages\monai\utils\deprecate_utils.py:321: FutureWarning: monai.transforms.croppad.dictionary CropForegroundd.__init__:allow_smaller: Current default value of argument `allow_smaller=True` has been deprecated since version 1.2. It will be changed to `allow_smaller=False` in version 1.5.
  warn_deprecated(argname, msg, warning_category)
100%|██████████| 268/268 [00:07<00:00, 34.13it/s]

Pixel_Counts: [[5.2627615e+08 3.5760586e+07]]
class_weights: tensor([0.0636, 0.9364])


In [4]:
print(pixel_counts[0][1])
print(pixel_counts[0][0])

35760586.0
526276150.0
